In [7]:
import os
os.chdir("/Users/Kuba/Library/CloudStorage/OneDrive-SGH/SGH/Rok IV/Analiza danych w czasie rzeczywistym/internet-of-trams") ### This needs to be replaced.

from internet_of_trams.frontend.trams_dasahboard_job_copy import TramsDashboardJob
from internet_of_trams.utils.get_config import get_config
import dash
from dash import html, dcc
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc

In [14]:
%%writefile internet_of_trams/frontend/app.py
import os, sys
sys.path.append(os.path.abspath(os.path.join(__file__, '..', '..', '..')))

from internet_of_trams.frontend.trams_dasahboard_job_copy import TramsDashboardJob
from internet_of_trams.utils.get_config import get_config
import dash
from dash import html, dcc
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
from tortoise import Tortoise, run_async

config = get_config()

job = TramsDashboardJob(
    ztm_api_key=config["API_KEY"],
    database_password=config["DATABASE_PASSWORD"])

run_async(job.get_static_data())

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("ZTM communication in Warsaw (trams)"),    
    html.Iframe(id="map_ZTM", width="100%", height='450'),    
    dbc.Col([
        dcc.Dropdown(
            id='line-selector',
            options=[{'label': str(line), 'value': line} for line in config["LINES"]],
            value=None, 
            multi=True
        ),
        html.Br(),
        html.Label("Zoom Level"),
        dcc.Slider(
            id='zoom-slider',
            min=-0,
            max=5,
            step=0.5,
            value=0,
            marks={i: str(i) for i in range(0, 6, 5)}
        ),
        html.Br(),
        html.Label("X Position"),
        dcc.Slider(
            id='x-slider',
            min=-20,
            max=20,
            step=0.1,
            value=0,
            marks={i: str(i) for i in range(-20, 21, 5)}
        ),
        html.Br(),
        html.Label("Y Position"),
        dcc.Slider(
            id='y-slider',
            min=-20,
            max=20,
            step=0.1,
            value=0,
            marks={i: str(i) for i in range(-20, 21, 5)}
        )
    ], width=7),
    dcc.Interval(
        id='interval-component',
        interval=10*1000,  # 10 seconds
        n_intervals=0
    )
])

@app.callback(
    Output("map_ZTM", "srcDoc"),
    [Input('interval-component', 'n_intervals'),
     Input("line-selector", "value"),
     Input("zoom-slider", "value"),
     Input("x-slider", "value"),
     Input("y-slider", "value")]
)
def update_map(n_intervals, selected_lines_dropdown, zoom, longitude_shift, latitude_shift):
    if selected_lines_dropdown is None:
        lines = []
    else:
        lines = selected_lines_dropdown
    
    return job.get_map_html(lines, zoom, longitude_shift, latitude_shift)

if __name__ == '__main__':
    app.run_server(debug=False, port = 3000)


Writing internet_of_trams/frontend/app.py


In [11]:
import dash
from dash import html, dcc
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc

def generate_map_html(selected_lines, zoom, x, y):
    return job.show_trams(selected_lines, zoom, x, y).get_root().render()

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("ZTM communication in Warsaw (trams)"),    
    html.Iframe(id="map_ZTM", width="100%", height='450'),    
    dbc.Col([
        dcc.Dropdown(
            id='line-selector',
            options=[{'label': str(line), 'value': line} for line in config["LINES"]],
            value=None, 
            multi=True
        ),
        html.Br(),
        dcc.Slider(
            id='zoom-slider',
            min=-0,
            max=5,
            step=0.5,
            value=0,
            marks={i: str(i) for i in range(0, 6, 5)}
        ),
        dcc.Slider(
            id='x-slider',
            min=-20,
            max=20,
            step=0.1,
            value=0,
            marks={i: str(i) for i in range(-20, 21, 5)}
        ),
        dcc.Slider(
            id='y-slider',
            min=-20,
            max=20,
            step=0.1,
            value=0,
            marks={i: str(i) for i in range(-20, 21, 5)}
        )
    ], width=7),
    dcc.Interval(
        id='interval-component',
        interval=10*1000,  # 10 seconds
        n_intervals=0
    )
])

@app.callback(
    Output("map_ZTM", "srcDoc"),
    [Input('interval-component', 'n_intervals'),
     Input("line-selector", "value"),
     Input("zoom-slider", "value"),
     Input("x-slider", "value"),
     Input("y-slider", "value")]
)
def update_map(n_intervals, selected_lines_dropdown, zoom, x, y):
    if selected_lines_dropdown is None:
        return generate_map_html([], zoom = zoom, x = x, y = y)
    
    return generate_map_html(selected_lines_dropdown, zoom = zoom, x = x, y = y)

if __name__ == '__main__':
    app.run_server(debug=True)


---------------------------------------------------------------------------
DependencyException                       Traceback (most recent call last)
DependencyException: "dash" is registered but the path requested is not valid.
The path requested: "dash-renderer/build/dash_renderer.min.js"
List of registered paths: defaultdict(<class 'set'>, {'dash': {'html/dash_html_components.min.js', 'dash-renderer/build/dash_renderer.dev.js.map', 'dash-renderer/build/dash_renderer.dev.js', 'dcc/async-mathjax.js', 'dcc/async-markdown.js', 'dash_table/async-highlight.js', 'dash_table/async-table.js', 'dcc/async-graph.js', 'dcc/async-highlight.js', 'html/dash_html_components.min.js.map', 'dcc/dash_core_components.js', 'dash_table/bundle.js.map', 'dash_table/async-export.js', 'dcc/async-highlight.js.map', 'dcc/async-datepicker.js.map', 'dcc/async-graph.js.map', 'deps/react@16.14.0.js', 'dcc/dash_core_components-shared.js.map', 'dash_table/async-export.js.map', 'dcc/dash_core_components-shared.js', '

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
Cell In[11], line 64, in update_map(
    n_intervals=0,
    selected_lines_dropdown=None,
    zoom=0,
    x=0,
    y=0
)
     54 @app.callback(
     55     Output("map_ZTM", "srcDoc"),
     56     [Input('interval-component', 'n_intervals'),
   (...)
     61 )
     62 def update_map(n_intervals, selected_lines_dropdown, zoom, x, y):
     63     if selected_lines_dropdown is None:
---> 64         return generate_map_html([], zoom = zoom, x = x, y = y)
        zoom = 0
        x = 0
        y = 0
     66     return generate_map_html(selected_lines_dropdown, zoom = zoom, x = x, y = y)

Cell In[11], line 7, in generate_map_html(selected_lines=[], zoom=0, x=0, y=0)
      6 def generate_map_html(selected_lines, zoom, x, y):
----> 7     return job.show_trams(selected_lines, zoom, x, y).get_root().render()
        selected_lines = []
        zo